### 自注意力

自注意力机制指的是对于一个序列样本，将其每一个token即看作键也看做值（用来表示序列的键值对）

同时将序列中每一个 token 视为对该序列的一个查询

```
序列：(batch_size, seq_len, embed_size)

键/值：(batch_size, seq_len, embed_size)，其中 seq_len = num_pairs，embed_size = key_value_size

查询：(batch_size, seq_len, embed_size)，其中 seq_len = num_query，embed_size = query_size
```

也就是说，自注意力中 Q、K、V 矩阵的形状一样

<br>

### 自注意力输出的意义

自注意力的输出形状：(batch_size, seq_len, embed_size)

该输出相当于对原序列做了个变换，仍然用来表示序列信息

相比原来的序列来说，每一个 token 都考虑了整个序列的上下文信息

相当于获取了每一个 token 在特定的上下文中的独特意思（同一个词在不同的话中表达的意思可能不同）

<br>

### 位置编码的必要性

上述的自注意力机制中，其实有一定的缺陷。

键值对（每一个 token）在使用的时候其实是无序的，而序列本身的每一个 token 是有序的

我们需要为一个序列中每一个 token 添加其位置信息

该操作（嵌入位置编码）在序列进行 embedding 后就进行

嵌入位置信息后再进行自注意力的计算

<br>

### 经典正弦余弦位置编码（固定编码）

位置编码公式如下：

$$
\text{PE}_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i / d_{model}}}\right)
$$

$$
\text{PE}_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i / d_{model}}}\right)
$$

其中：

- $pos$ 表示序列中的位置（每一个 token 在序列中的位置索引）

- $i$ 表示维度索引（用来表示一个 token 的向量的各个维度）

- $d_{model}$ 是嵌入向量维度

In [11]:
import torch
from torch import nn

In [ ]:
class PositionEncoder(nn.Module):
    def __init__(self):
        super().__init__()


    # x 的形状：(batch_size, seq_len, embed_size)
    def forward(self, x):
        batch_size, seq_len, embed_size = x.shape

        # 获取表示矩阵中每一个元素所属的 token 在序列中的索引
        pos_map = torch.arange(seq_len).unsqueeze(1)      # (seq_len, 1)
        pos_map = pos_map.expand(seq_len, embed_size)     # (seq_len, embed_size)


        # 获取表示矩阵中每个元素在表示 token 的向量中的维度
        dim_map = torch.arange(embed_size).unsqueeze(0)      # (1, embed_size)
        dim_map = dim_map.expand(seq_len, embed_size)        # (seq_len, embed_size)

        # 计算位置编码矩阵
        angle = pos_map / torch.pow(10000, 2 * dim_map / embed_size)    # (seq_len, embed_size)
        pe = torch.zeros(seq_len, embed_size)       # (seq_len, embed_size)
        pe[:, 0::2] = torch.sin(angle[:, 0::2])     # dim 为偶数，使用 sin
        pe[:, 1::2] = torch.cos(angle[:, 1::2])     # dim 为奇数，使用 cos

        # 每个样本对应位置的位置编码是一致的
        pe = pe.unsqueeze(0).expand(batch_size, seq_len, embed_size)    # (batch_size, seq_len, embed_size)

        return x + pe   # 嵌入位置编码

<br>

### 含位置编码的自注意力代码实现

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, hiden_size):
        super().__init__()
        self.hiden_size = torch.tensor(hiden_size)
        self.position_encoder = PositionEncoder()
        self.linear_q = nn.Linear(embed_size, hiden_size)
        self.linear_k = nn.Linear(embed_size, hiden_size)
        self.linear_v = nn.Linear(embed_size, hiden_size)
        self.softmax = nn.Softmax(dim=2)


    # x 的形状：(batch_size, seq_len, embed_size)
    def forward(self, x):
        x = self.position_encoder(x)     # (batch_size, seq_len, embed_size)
        query = self.linear_q(x)         # (batch_size, seq_len, hiden_size)
        key = self.linear_q(x)           # (batch_size, seq_len, hiden_size)
        value = self.linear_q(x)         # (batch_size, seq_len, hiden_size)
        score = torch.bmm(query, key.permute(0, 2, 1)) / torch.sqrt(self.hiden_size)   # (batch_size, seq_len, seq_len)
        weight = self.softmax(score)     # (batch_size, seq_len, seq_len)
        return torch.bmm(weight, value)  # (batch_size, seq_len, hiden_size)